<a href="https://colab.research.google.com/github/osaeed-ds/HybridSearch/blob/main/HybridSearch_ShipmentData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/awesome-astra/docs/blob/main/docs/pages/tools/notebooks/Retrieval_Augmented_Generation_(for_AI_Chatbots).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Getting Started with this notebook**

- Create a new vector search enabled database in Astra. [astra.datastax.com](https://astra.datastax.com)
- Create a token with permissions to create tables (eg: Database Administrator)
- Download your secure-connect-bundle zip file.

# **Setup**

This section installs dependencies, handles the main imports, and connects to the database with the provided details.

In [1]:
!pip install cassandra-driver sentence-transformers pandas transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=1264c8870d4be1f7a2b69cd4621f811bc06b9ea246077390e4927e4d7ad0b3d2
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory
from cassandra.query import SimpleStatement

from getpass import getpass

In [115]:
# Download the sample data, which is based on the Hazmat Dataset at https://geography.wisc.edu/hazardouswaste/downloads.html
!curl https://raw.githubusercontent.com/osaeed-ds/HybridSearch/main/waste_shipments.csv --output waste_shipments.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  503k  100  503k    0     0  1570k      0 --:--:-- --:--:-- --:--:-- 1572k


In [3]:
# Input your Astra DB token string, the one starting with "AstraCS:..."
ASTRA_DB_TOKEN_BASED_PASSWORD = getpass('Your Astra DB Token ("AstraCS:..."): ')

Your Astra DB Token ("AstraCS:..."): ··········


In [4]:
#Change your keyspace name if needed
my_ks='vector_preview'

In [5]:
# Upload your Secure Connect Bundle zipfile:
import os
from google.colab import files

print('Please upload your Secure Connect Bundle')
uploaded = files.upload()
if uploaded:
    astraBundleFileTitle = list(uploaded.keys())[0]
    scb_path = os.path.join(os.getcwd(), astraBundleFileTitle)
else:
    raise ValueError(
        'Cannot proceed without Secure Connect Bundle. Please re-run the cell.'
    )

Please upload your Secure Connect Bundle


Saving secure-connect-osaeed-vector.zip to secure-connect-osaeed-vector.zip


In [71]:
# Uses the secure-connect-bundle and the provided token to connect to Cassandra

cloud_config= {
  'secure_connect_bundle': scb_path
}
auth_provider = PlainTextAuthProvider('token', ASTRA_DB_TOKEN_BASED_PASSWORD)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider, protocol_version=4)
session = cluster.connect()
session.set_keyspace(my_ks)
session

# **Load Data**

This section creates the table and indexes, prepares the sample data, and loads it into the DB

In [110]:
# Run this to drop the table and indexes before starting over
session.execute("DROP TABLE IF EXISTS shipments_table")

In [111]:
# Create the table
mktable_cql = """CREATE TABLE shipments_table (
shipment_id int PRIMARY KEY,
shipper_id text,
shipper_name text,
shipper_address text,
shipper_city text,
shipper_state text,
shipper_zip text,
shipper_latitude float,
shipper_longitude float,
waste_description text,
waste_epa_codes text,
weight float,
minilm vector<float, 384>
);
"""




session.execute(mktable_cql);


In [112]:
# create the indexes, including standard SAI, SAI with Term Search, and Vector
indexes = [
    "CREATE CUSTOM INDEX ON shipments_table(shipper_state) USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'",
    "CREATE CUSTOM INDEX ON shipments_table(shipper_latitude) USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'",
    "CREATE CUSTOM INDEX ON shipments_table(shipper_longitude) USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'",
    "CREATE CUSTOM INDEX ON shipments_table(weight) USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'",
    """
    CREATE CUSTOM INDEX ON shipments_table(minilm)
    USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'
    WITH OPTIONS = { 'similarity_function': 'dot_product' }""",
    """
    CREATE CUSTOM INDEX ON shipments_table(waste_epa_codes)
    USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'
    WITH OPTIONS = {
    'index_analyzer': '{
    "tokenizer" : {"name" : "standard"},
    "filters" : [{"name" : "porterstem"},{"name" : "lowercase",	"args": {}}]
    }'}""",
    """
    CREATE CUSTOM INDEX ON shipments_table(waste_description)
    USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'
    WITH OPTIONS = {
    'index_analyzer': '{
    "tokenizer" : {"name" : "standard"},
    "filters" : [{"name" : "porterstem"},{"name" : "lowercase",	"args": {}}]
    }'};"""

]
for index in indexes:
  session.execute(index)



In [116]:
# Here, the waste_shipments.csv file is loaded into a Pandas dataframe

import pandas as pd

shipments_list = pd.read_csv('waste_shipments.csv')

shipments_list

,shipment_id,shipper_id,shipper_name,shipper_address,shipper_city,shipper_state,shipper_zip,shipper_latitude,shipper_longitude,waste_description,waste_epa_codes,weight
0,1,AZD009015389,ROMIC ENVIRONMENTAL TECHNOLOGIES,6760 W. ALLISON ROAD,CHANDLER,AZ,85226,33.288191,-111.957002,WASTE ACETYLENE,D001,54.432
1,2,AZD009015389,ROMIC ENVIRONMENTAL TECHNOLOGIES,6760 W. ALLISON ROAD,CHANDLER,AZ,85226,33.288191,-111.957002,"WASTE ACETYLENE, DISSOLVED",D001,18.144
2,3,AZD009015389,ROMIC ENVIRONMENTAL TECHNOLOGIES,6760 W. ALLISON ROAD,CHANDLER,AZ,85226,33.288191,-111.957002,WASTE ACETYLENE,D001,52.164
3,4,AZD009015389,ROMIC ENVIRONMENTAL TECHNOLOGIES,6760 W. ALLISON ROAD,CHANDLER,AZ,85226,33.288191,-111.957002,"WASTE CARBON MONOXIDE, COMPRESSED",D001,4.536
4,5,AZD009015389,ROMIC ENVIRONMENTAL TECHNOLOGIES,6760 W. ALLISON ROAD,CHANDLER,AZ,85226,33.288191,-111.957002,"WASTE HYDROGEN, COMPRESSED",D001,22.680
...,...,...,...,...,...,...,...,...,...,...,...,...
3244,3246,NCD980842132,ECOFLO INC,2750 PATTERSON STREET,GREENSBORO,NC,27407,36.058130,-79.837126,SILVER NITRATE AND WATER,D011,24.948
3245,3247,IND000646943,TRADEBE TREATMENT AND RECYCLING,4343 KENNEDY AVENUE,EAST CHICAGO,IN,46312,41.633689,-87.461683,OXIDIZING LIQUID,D011,80.000
3246,3248,IND000646943,TRADEBE TREATMENT AND RECYCLING,4343 KENNEDY AVENUE,EAST CHICAGO,IN,46312,41.633689,-87.461683,CORROSIVE LIQUID,D011,337.000
3247,3249,NYD049178296,RADIAC RESEARCH CORP.,33 S 1ST STREET,BROOKLYN,NY,11211,40.715280,-73.965564,"WASTE ENVIRONMENTALLY HAZARDOUS SUBSTANCE, LIQUID",D011; D018,716.688


In [117]:
# Download the all-MiniLM-L6-v2 from HuggingFace, and configure it to infer on the CPU (as opposed to GPU)
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')

In [118]:
#If our largest column size is small enough, we don't need to worry about chunking
max_size_desc = shipments_list['waste_description'].apply(lambda x: len(str(x))).max()
print(max_size_desc)

221


In [119]:
# This block calculates the embedding and loads the rows from the dataframe
insert_pstmt = session.prepare("""
INSERT INTO shipments_table
(shipment_id, shipper_id,	shipper_name,	shipper_address,	shipper_city,	shipper_state,	shipper_zip,	shipper_latitude,	shipper_longitude,	waste_description,	waste_epa_codes,	weight, minilm)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""")



#Note that this is a naive loading technique and will be too slow for large data
#For "real" data loads, always use an async approach
#for this dataset will take about 5 minutes to execute
for id, row in shipments_list.iterrows():
  emb = model.encode(row.waste_description).tolist()
  session.execute(insert_pstmt, [row.shipment_id, row.shipper_id,	row.shipper_name,	row.shipper_address,	row.shipper_city,	row.shipper_state,	row.shipper_zip,	row.shipper_latitude,	row.shipper_longitude,	row.waste_description,	row.waste_epa_codes,	row.weight, emb])


# **Queries**

This shows queries that use text-search, semantic-search, and combinations of both.

In [120]:
#Perform a query using a simple term against the waste_description column
textsearch_query = session.execute(
    """
    SELECT shipper_name, waste_description, waste_epa_codes, weight
    FROM shipments_table
    WHERE waste_description : 'PROPANE' LIMIT 25
    """)

results = pd.DataFrame(textsearch_query)
results

,shipper_name,waste_description,waste_epa_codes,weight
0,ROMIC ENVIRONMENTAL TECHNOLOGIES,WASTE PROPANE,D001,13.608000
1,"CYCLE CHEMICAL, INC.",PROPANE,D001,5097.556641
2,ROMIC ENVIRONMENTAL TECHNOLOGIES,WASTE PROPANE,D001,61.236000
3,ROMIC ENVIRONMENTAL TECHNOLOGIES,WASTE PROPANE,D001,147.419998
4,ROMIC ENVIRONMENTAL TECHNOLOGIES,WASTE PROPANE,D001,81.648003
5,ROMIC ENVIRONMENTAL TECHNOLOGIES,WASTE PROPANE,D001,27.216000
6,ROMIC ENVIRONMENTAL TECHNOLOGIES,WASTE PROPANE,D001,22.226400
7,GENERAL ENVIRONMENTAL MANAGEMENT,"PROPANE, CYLINDERS",D001,149.688004
8,"CYCLE CHEMICAL, INC.",PROPANE,D001,1.000000
9,CYCLE CHEM OF LEWISBERRY INC,PROPANE (TORCH SIZE),D001,996.559204


In [121]:
#Now, embed a query string as a vector instead, and read out the closest rows to the query vector
to_embed = "Corrosive Solid Inorganic"
query_vec = model.encode(to_embed).tolist()
semantic_query = session.execute(
    f"""
    SELECT shipper_name, waste_description, waste_epa_codes, weight
    FROM shipments_table ORDER BY minilm ANN OF {query_vec} LIMIT 25
    """)
results = pd.DataFrame(semantic_query)
results

,shipper_name,waste_description,waste_epa_codes,weight
0,CLEAN HARBORS INC,CORROSIVE INORGANIC SOLIDS,D002,7130.000000
1,"AMERICAN ENVIRONMENTAL SERVICES, INC.","CORROSIVE SOLID, BASIC, INORGANIC",D002; D004; D005; D006; D007; D008; D009; D010...,562.463989
2,"AMERICAN ENVIRONMENTAL SERVICES, INC.","CORROSIVE SOLIDS, BASIC, INORGANIC",D002; D007,420.487213
3,CHEMICAL POLLUTION CONTROL INC,"CORROSIVE SOLID, ACIDIC, INORGANIC",D002; D008,99.792000
4,ONYX ENVIRONMENTAL SERVICES,"CORROSIVE SOLID, ACIDIC, INORGANIC",D002; D006,567.000000
5,"AMERICAN ENVIRONMENTAL SERVICES, INC.","CORROSIVE SOLID, ACIDIC, INORGANIC",D002; D004; D005; D006; D007; D008; D009; D010...,694.007996
6,"LWD, INC - SHAR CAL FACILITY","CORROSIVE SOLID, ACIDIC, INORGANIC",D001; D003; D005; D007; D008,4.082400
7,"LWD, INC - SHAR CAL FACILITY","CORROSIVE SOLID, ACIDIC, INORGANIC",D002; D004; D005; D006; D007; D008; D010; D011,7649.963867
8,"AMERICAN ENVIRONMENTAL SERVICES, INC.","CORROSIVE SOLIDS, TOXIC, INORGANIC",D001; D004; D008; D009,5.443200
9,"AMERICAN ENVIRONMENTAL SERVICES, INC.","CORROSIVE SOLIDS, ACIDIC, INORGANIC",D002; D003; D005; D007; D008,226.800003


In [122]:
#Metadata Filtering example:
#Here, a semantic query is combined with term search and SAI range predicates
#Any non-vector, non-full-text column in the table is metadata for the vector.
new_embed = "Corrosive Solid Inorganic"
query_vec = model.encode(new_embed).tolist()
hybrid_query = session.execute(
    f"""
    SELECT shipper_name, waste_description, waste_epa_codes, weight, shipper_latitude, shipper_longitude, shipper_state
    FROM shipments_table
    WHERE shipper_latitude > 35 and shipper_latitude < 40
    AND waste_epa_codes : 'D008'
    ORDER BY minilm ANN OF {query_vec} LIMIT 25
    """)
funstuff = pd.DataFrame(hybrid_query)
funstuff

,shipper_name,waste_description,waste_epa_codes,weight,shipper_latitude,shipper_longitude,shipper_state
0,"AMERICAN ENVIRONMENTAL SERVICES, INC.","CORROSIVE SOLID, BASIC, INORGANIC",D002; D004; D005; D006; D007; D008; D009; D010...,562.463989,39.628757,-79.987511,WV
1,"AMERICAN ENVIRONMENTAL SERVICES, INC.","CORROSIVE SOLID, ACIDIC, INORGANIC",D002; D004; D005; D006; D007; D008; D009; D010...,694.007996,39.628757,-79.987511,WV
2,"LWD, INC - SHAR CAL FACILITY","CORROSIVE SOLID, ACIDIC, INORGANIC",D001; D003; D005; D007; D008,4.082400,37.049389,-88.405045,KY
3,"LWD, INC - SHAR CAL FACILITY","CORROSIVE SOLID, ACIDIC, INORGANIC",D002; D004; D005; D006; D007; D008; D010; D011,7649.963867,37.049389,-88.405045,KY
4,"AMERICAN ENVIRONMENTAL SERVICES, INC.","CORROSIVE SOLIDS, TOXIC, INORGANIC",D001; D004; D008; D009,5.443200,39.628757,-79.987511,WV
5,"AMERICAN ENVIRONMENTAL SERVICES, INC.","CORROSIVE SOLIDS, ACIDIC, INORGANIC",D002; D003; D005; D007; D008,226.800003,39.628757,-79.987511,WV
6,"LWD, INC - SHAR CAL FACILITY",CORROSIVE SOLID,D002; D004; D005; D006; D007; D008; D010; D011,5033.145508,37.049389,-88.405045,KY
7,"LWD, INC - SHAR CAL FACILITY","CORROSIVE LIQUID, BASIC, INORGANIC",D002; D004; D005; D006; D007; D008; D010; D011,15775.754883,37.049389,-88.405045,KY
8,"AMERICAN ENVIRONMENTAL SERVICES, INC.","CORROSIVE LIQUIDS, BASIC, INORGANIC",D002; D004; D005; D006; D007; D008; D009; D010...,4801.809570,39.628757,-79.987511,WV
9,"LWD, INC - SHAR CAL FACILITY","CORROSIVE LIQUIDS, BASIC, INORGANIC",D002; D004; D005; D006; D007; D008; D009; D010...,65999.710938,37.049389,-88.405045,KY
